# Practice notebook for confidence intervals using NHANES data

This notebook will give you the opportunity to practice working with confidence intervals using the NHANES data.

You can enter your code into the cells that say "enter your code here", and you can type responses to the questions into the cells that say "Type Markdown and Latex".

Note that most of the code that you will need to write below is very similar to code that appears in the case study notebook.  You will need to edit code from that notebook in small ways to adapt it to the prompts below.

To get started, we will use the same module imports and read the data in the same way as we did in the case study:

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm

da = pd.read_csv("nhanes_2015_2016.csv")

## Question 1

Restrict the sample to women between 35 and 50, then use the marital status variable [DMDMARTL](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#DMDMARTL) to partition this sample into two groups - women who are currently married, and women who are not currently married.  Within each of these groups, calculate the proportion of women who have completed college.  Calculate 95% confidence intervals for each of these proportions.

In [2]:
""" 
RIAGENDR == 2 # code for women
(da.RIDAGEYR >= 35) & (da.RIDAGEYR <= 50)   #restrict age range between 35 and 50
(da.DMDMARTL == 1)   # group data by Married
(da.DMDMARTL != 1)   # group data by not  Married
DMDEDUC2  code for education level of completed college = 5 
"""
# making datafram for women with age (35,50)
dw = da[(da.RIAGENDR == 2) & (da.RIDAGEYR >= 35) & (da.RIDAGEYR <= 50)].copy()
dw['DMDMARTLx'] = dw.DMDMARTL.replace({1 : "Married", 
                                       2 : "Not_Married", 
                                       3 : "Not_Married", 
                                       4 : "Not_Married", 
                                       5 : "Not_Married", 
                                       6 : "Not_Married", 
                                       77: "Not_Married"
                                       })

dw['DMDEDUC2x'] = dw.DMDEDUC2.replace({1 : "Other", 
                                       2 : "Other", 
                                       3 : "Other", 
                                       4 : "Other", 
                                       5 : "College_graduate", 
                                       7 : "Other", 
                                       9 : "Other"
                                       })


In [4]:
# check  for non values 
(dw.DMDMARTLx.value_counts(), dw.DMDEDUC2x.value_counts())

(Married        449
 Not_Married    338
 Name: DMDMARTLx, dtype: int64,
 Other               553
 College_graduate    234
 Name: DMDEDUC2x, dtype: int64)

In [5]:
dw.groupby(['DMDMARTLx','DMDEDUC2x'])['DMDEDUC2x'].count().unstack()

DMDEDUC2x,College_graduate,Other
DMDMARTLx,,
Married,162,287
Not_Married,72,266


In [6]:
# convert values back to numeric so we could calculate the mean
dw['DMDEDUC2x'] = dw.DMDEDUC2x.replace({"Other":0, "College_graduate":1})

In [7]:

# calculate p hat (proportion of college graduted in poth married and not married)

dz = dw.groupby(dw.DMDMARTLx).agg({"DMDEDUC2x": [lambda x : np.mean( x == 1 ), np.size]})
dz.columns = ["Proportion", "Total_n"] # The default column names are unclear, so we replace them here
dz

,Proportion,Total_n
DMDMARTLx,,
Married,0.360802,449
Not_Married,0.213018,338


In [8]:
# calculate it with numpy 
p = dz.Proportion.Married # Married proportion
n = dz.Total_n.Married # Total number of Married
se_married = np.sqrt(p * (1 - p) / n)
print(se_married)

p = dz.Proportion.Not_Married # Not Married proportion
n = dz.Total_n.Not_Married # Total number of Not Married
se_not_married = np.sqrt(p * (1 - p) / n)
print(se_not_married)

0.02266360248455356
0.022270605048202215


In [9]:
# CI for all population colleage grad in married women
p = dz.Proportion.Married # Married proportion
lcb = p - 1.96 * se_married  
ucb = p + 1.96 * se_married 
print(lcb, ucb)

0.3163811208674688 0.4052224426069187


In [10]:
# CI for all population Not marreid women with colleage 

p = dz.Proportion.Not_Married # Not Married proportion
lcb = p - 1.96 * se_not_married
ucb = p + 1.96 * se_not_married
print(lcb, ucb)

0.1693673655848136 0.25666813737376626


In [13]:
# check our calculations by statsmodle
Married_por = sm.stats.proportion_confint(dz.Total_n.Married * dz.Proportion.Married,dz.Total_n.Married)
Not_married_por = sm.stats.proportion_confint(dz.Total_n.Not_Married * dz.Proportion.Not_Married,dz.Total_n.Not_Married)

(Married_por,Not_married_por)

((0.31638193710753626, 0.4052216263668512),
 (0.16936816767089768, 0.2566673352876822))

__Q1a.__ Identify which of the two confidence intervals is wider, and explain why this is the case. 

they a bout the same because the level of confidence is the same with smiller sample sizes also

__Q1b.__ Write 1-2 sentences summarizing these findings for an audience that does not know what a confidence interval is (the goal here is to report the substance of what you learned about how marital status and educational attainment are related, not to teach a person what a confidence interval is).

with % 95  confidence we astimate that  ( % 31.63 : % 40.52 ) of all married women at age (35,50) have college degrees, and (% 16.93 : % 25.66) of all  Not married women at the same age have college degrees
form this we could say that having college degrees are associated with Marrage rate.

## Question 2

Construct a 95% confidence interval for the proportion of smokers who are female. Construct a 95% confidence interval for the proportion of smokers who are male. Construct a 95% confidence interval for the **difference** between those two gender proportions.

In [14]:
da["SMQ020x"] = da.SMQ020.replace({1: "Yes", 2: "No", 7: np.nan, 9: np.nan})  # np.nan represents a missing value
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

We can now [tabulate](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.crosstab.html) the numbers of female and male smokers and non-smokers:

In [15]:
dx = da[["SMQ020x", "RIAGENDRx"]].dropna()  # dropna drops cases where either variable is missing
pd.crosstab(dx.SMQ020x, dx.RIAGENDRx)

RIAGENDRx,Female,Male
SMQ020x,,
No,2066,1340
Yes,906,1413


In [16]:
Females_pro = sm.stats.proportion_confint(906,906+2066)
Males_pro = sm.stats.proportion_confint(1413,1413+1340)
(Females_pro,Males_pro) 

((0.2882949879861214, 0.32139545615923526),
 (0.49458749263718593, 0.5319290347874418))

In [21]:
# calculate difference between two proprtions 
n1 = (906 + 2066) 
n2 = (1413 + 1340)
p1 = 906 / n1
p2 = 1413 / n2
Se1 = np.sqrt(p1 * (1 - p1) / n1)
Se2 = np.sqrt(p2 * (1-p2) / n2)
Se_diff = np.sqrt(Se1**2 + Se2**2)
Se_diff

0.012729881381407434

In [23]:
d = p1 - p2 
lcb = d - 2*Se_diff
ucb = d + 2*Se_diff
(lcb,ucb)


(-0.2338728044024504, -0.18295327887682067)

__Q2a.__ Why might it be relevant to report the separate gender proportions **and** the difference between the gender proportions?

because we give ranges not specified numbers so,
the separate gender proportions gives you the a whole range for each gender
and differece summarizes the proportions differece between the two ranges.

__Q2b.__ How does the **width** of the confidence interval for the difference between the gender proportions compare to the widths of the confidence intervals for the separate gender proportions?

the width of the difference between the two porporations is wider compared to the width of separated porporations for each gender.

## Question 3

Construct a 95% interval for height ([BMXHT](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BMX_I.htm#BMXHT)) in centimeters.  Then convert height from centimeters to inches by dividing by 2.54, and construct a 95% confidence interval for height in inches.  Finally, convert the endpoints (the lower and upper confidence limits) of the confidence interval from inches to back to centimeters   

In [ ]:
# enter your code here

__Q3a.__ Describe how the confidence interval constructed in centimeters relates to the confidence interval constructed in inches.

## Question 4

Partition the sample based on 10-year age bands, i.e. the resulting groups will consist of people with ages from 18-28, 29-38, etc. Construct 95% confidence intervals for the difference between the mean BMI for females and for males within each age band.

In [ ]:
# enter your code here

__Q4a.__ How do the widths of these confidence intervals differ?  Provide an explanation for any substantial diferences in the confidence interval widths that you see.

## Question 5

Construct a 95% confidence interval for the first and second systolic blood pressure measures, and for the difference between the first and second systolic blood pressure measurements within a subject.

In [ ]:
# enter code here

__Q5a.__ Based on these confidence intervals, would you say that a difference of zero between the population mean values of the first and second systolic blood pressure measures is consistent with the data?

__Q5b.__ Discuss how the width of the confidence interval for the within-subject difference compares to the widths of the confidence intervals for the first and second measures.

## Question 6

Construct a 95% confidence interval for the mean difference between the average age of a smoker, and the average age of a non-smoker.

In [ ]:
# insert your code here

__Q6a.__ Use graphical and numerical techniques to compare the variation in the ages of smokers to the variation in the ages of non-smokers.  

In [1]:
# insert your code here

__Q6b.__ Does it appear that uncertainty about the mean age of smokers, or uncertainty about the mean age of non-smokers contributed more to the uncertainty for the mean difference that we are focusing on here?